In [1]:
%cd mmdetection

/mnt/nas2/users/eslim/workspace/comp/mmdetection


In [2]:

import os
from natsort import natsorted
from mmdet.apis import DetInferencer
from PIL import Image
import json
from mmengine import ConfigDict
from mmengine.config import Config, DictAction
import copy


os.environ["CUDA_VISIBLE_DEVICES"]="2"

device = 'cuda:0'
config_path = "projects/CO-DETR/configs/codino/co_dino_5scale_r50_lsj_8xb2_1x_resnet101.py"
checkpoint = "./work_dirs/resnet101/refine/iter_118000.pth"
cfg = Config.fromfile(config_path)

image_path_prefix = "./data/all_dataset/roboflow_data/train/"
save_prefix = "./data/all_dataset/anno/"

extra_image_num = 1500
pseudo_thres = 0.60

In [3]:
inferencer = DetInferencer(cfg, checkpoint, device)


Loads checkpoint by local backend from path: ./work_dirs/resnet101/refine/iter_118000.pth


/mnt/nas2/users/eslim/workspace/comp/mmdetection/projects/CO-DETR/codetr/transformer.py:1327: UserWarning: If you want to reduce GPU memory usage,                               please install fairscale by executing the                               following command: pip install fairscale.
  following command: pip install fairscale.')


09/13 02:18:51 - mmengine - INFO - 
rpn_conv.weight - torch.Size([256, 256, 3, 3]): 
NormalInit: mean=0, std=0.01, bias=0 
 
09/13 02:18:51 - mmengine - INFO - 
rpn_conv.bias - torch.Size([256]): 
NormalInit: mean=0, std=0.01, bias=0 
 
09/13 02:18:51 - mmengine - INFO - 
rpn_cls.weight - torch.Size([9, 256, 1, 1]): 
NormalInit: mean=0, std=0.01, bias=0 
 
09/13 02:18:51 - mmengine - INFO - 
rpn_cls.bias - torch.Size([9]): 
NormalInit: mean=0, std=0.01, bias=0 
 
09/13 02:18:51 - mmengine - INFO - 
rpn_reg.weight - torch.Size([36, 256, 1, 1]): 
NormalInit: mean=0, std=0.01, bias=0 
 
09/13 02:18:51 - mmengine - INFO - 
rpn_reg.bias - torch.Size([36]): 
NormalInit: mean=0, std=0.01, bias=0 
 
09/13 02:18:52 - mmengine - INFO - 
bbox_head.fc_cls.weight - torch.Size([9, 1024]): 
NormalInit: mean=0, std=0.01, bias=0 
 
09/13 02:18:52 - mmengine - INFO - 
bbox_head.fc_cls.bias - torch.Size([9]): 
NormalInit: mean=0, std=0.01, bias=0 
 
09/13 02:18:52 - mmengine - INFO - 
bbox_head.fc_reg.we

/mnt/nas2/users/eslim/workspace/comp/mmdetection/mmdet/models/dense_heads/anchor_head.py:108: UserWarning: DeprecationWarning: `num_anchors` is deprecated, for consistency or also use `num_base_priors` instead
  warnings.warn('DeprecationWarning: `num_anchors` is deprecated, '


09/13 02:18:57 - mmengine - WARNING - Failed to search registry with scope "mmdet" in the "function" registry tree. As a workaround, the current "function" registry in "mmengine" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmdet" is a correct scope, or whether the registry is initialized.


/opt/conda/lib/python3.7/site-packages/mmengine/visualization/visualizer.py:196: UserWarning: Failed to add <class 'mmengine.visualization.vis_backend.LocalVisBackend'>, please provide the `save_dir` argument.
  warnings.warn(f'Failed to add {vis_backend.__class__}, '


In [4]:

with open('./data/all_dataset/roboflow_data/train/_annotations.coco.json', 'r') as file:
    ext_train_data = json.load(file)



In [5]:
len(ext_train_data)
ext_train_data["images"][-1]

{'id': 8859,
 'license': 1,
 'file_name': 'FLIR_08534_jpeg.rf.fffcd143bc6ee5bcdf4027161d4b0109.jpg',
 'height': 416,
 'width': 416,
 'date_captured': '2023-10-16T08:41:20+00:00'}

In [6]:
ext_train_data.keys()


dict_keys(['info', 'licenses', 'categories', 'images', 'annotations'])

In [7]:
ext_train_data["annotations"]

[{'id': 0,
  'image_id': 1,
  'category_id': 2,
  'bbox': [184, 181, 26.5, 29],
  'area': 768.5,
  'segmentation': [],
  'iscrowd': 0},
 {'id': 1,
  'image_id': 1,
  'category_id': 2,
  'bbox': [170, 174, 13, 14.5],
  'area': 188.5,
  'segmentation': [],
  'iscrowd': 0},
 {'id': 2,
  'image_id': 1,
  'category_id': 2,
  'bbox': [178, 164, 10.5, 12],
  'area': 126,
  'segmentation': [],
  'iscrowd': 0},
 {'id': 3,
  'image_id': 2,
  'category_id': 2,
  'bbox': [18, 186, 82, 56],
  'area': 4592,
  'segmentation': [],
  'iscrowd': 0},
 {'id': 4,
  'image_id': 2,
  'category_id': 2,
  'bbox': [90, 185, 36.5, 37.5],
  'area': 1368.75,
  'segmentation': [],
  'iscrowd': 0},
 {'id': 5,
  'image_id': 2,
  'category_id': 2,
  'bbox': [121, 183, 23.5, 31],
  'area': 728.5,
  'segmentation': [],
  'iscrowd': 0},
 {'id': 6,
  'image_id': 2,
  'category_id': 2,
  'bbox': [173, 189, 17, 18],
  'area': 306,
  'segmentation': [],
  'iscrowd': 0},
 {'id': 7,
  'image_id': 3,
  'category_id': 2,
  'bbox

In [8]:
total_num=0
ret_list=[]
for idx,info in enumerate(ext_train_data["annotations"]):
    file_name = ext_train_data["images"][info["image_id"]]["file_name"]
    image_path = os.path.join(image_path_prefix,file_name)
    results = inferencer(image_path)
    
    if total_num>extra_image_num:
        break
    total_num+=1
    for ret_idx,bbox in enumerate(results["predictions"][0]["bboxes"]):
        if results['predictions'][0]["scores"][ret_idx] > pseudo_thres:
            min_x, min_y, max_x, max_y = bbox
            ret = {
                    "image_id" : file_name,
                    "category_id" : results['predictions'][0]["labels"][ret_idx],
                    "bbox":[
                        min_x,
                        min_y,
                        max_x-min_x,
                        max_y-min_y
                    ],
                    "score":results['predictions'][0]["scores"][ret_idx],
                }
            ret_list.append(ret)
        
    


Output()

/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their 
associated APIs are an experimental feature and subject to change. Please do not use them for anything important 
until they are released as stable. (Triggered internally at  
/opt/conda/conda-bld/pytorch_1623448265233/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)

/opt/conda/lib/python3.7/site-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be 
removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This 
results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use 
torch.div(a, b, rounding_mode='floor'). (Triggered internally at  
/opt/conda/conda-bld/pytorch_1623448265233/work/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)

Output()

Output()

...

Output()

Output()

Output()

Output()

Output()

In [9]:
with open(os.path.join(save_prefix,"extra_pseudo_ann.txt"), "w") as file:
    json.dump(ret_list, file, indent=4)